In [ ]:
import itertools

import pandas as pd
import numpy as np
import statsmodels.api as sm
from matplotlib import pyplot as plt

import numba

from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn import datasets, linear_model
import statsmodels.api as sm
from scipy import stats


import warnings
warnings.filterwarnings('ignore')

In [ ]:
hmp = pd.read_csv("test_hapmap.txt", sep="\t", index_col=0)
hmp = hmp.iloc[:,10:] # HapMap files come with 10 unused columns
hmp = hmp.replace("N", np.nan)

trait = pd.read_csv("test_trait.txt", sep="\t", index_col=0)
trait.columns

# Get hmp columns with existing traits
hmp = hmp[hmp.columns[hmp.columns.isin(trait.index)]]
trait = trait.loc[hmp.columns]

hmp = hmp.T.sort_index()
trait = trait.sort_index()

# Simpler than the sklearn OrdinalEncoder
hmp = hmp.rank(axis=0, method="dense")-1
hmp

# 1. OLS GWAS

Currently only implemented for a single SNP but the result match TASSEL GLM output for the tested SNP.\
Thia is also likely to be too slow to calculate the test statistics for thousands of SNPs.

In [ ]:
# Might be interesting to use numba JIT with the custom applied functions
from numba import jit, njit, vectorize, float64

In [ ]:
# So far this is the simplest and fastest GWAS GLM method I could come up with
# It is significantly slower compared to any published method, but it is a start
def test(tmp):
    tmp = pd.concat([tmp, trait.iloc[:,0]], axis=1).dropna()
    #tmp.iloc[:,0] = OrdinalEncoder().fit_transform(tmp.iloc[:,0].values.reshape(-1, 1))
    tmp = sm.add_constant(tmp)
    return(sm.OLS(tmp.iloc[:,-1], tmp.iloc[:,:2]).fit().pvalues[1])

%time pvals = hmp.iloc[:,:1000].aggregate(test, axis=0)

In [ ]:
# The bare-bone approach works and produces the same p-values but it is much slower than the simpler statsmodels
# https://www.cluzters.ai/forums/topic/395/find-p-value-significance-in-scikit-learn-linear-regression?c=1597
def test(tmp):
    tmp = pd.concat([tmp, trait.iloc[:,0]], axis=1).dropna()
    tmp = sm.add_constant(tmp)
    lm = linear_model.LinearRegression()
    X, y = tmp.iloc[:,:2], tmp.iloc[:,-1]
    lm.fit(X, y)
    params = np.append(lm.intercept_,lm.coef_[-1])
    predictions = lm.predict(X)
    MSE = (sum((y-predictions)**2))/(len(X)-2) # mean square error
    sd_b = np.sqrt(MSE*(np.linalg.inv(np.dot(X.T,X)).diagonal()))
    ts_b = params[-1] / sd_b[-1] # t statistic for each
    p_values = 2*(1-stats.t.cdf(np.abs(ts_b),(len(X)-1)))
    return(np.round(p_values,5))

%time pvals = hmp.iloc[:,:1000].aggregate(test, axis=0)
#%time pvals = hmp.aggregate(test, axis=0)

In [ ]:
# I use the statsmodels.OLS single SNP prediction to assess other methods
# Merge genotype column and trait column in a single dataframe
data = pd.DataFrame(hmp.T.iloc[2,:]).join(trait.iloc[:,0])
data.columns = ["SNP", "trait"]
data["SNP"] = data["SNP"].replace("N", np.nan)
data = data.dropna()

from sklearn.preprocessing import OrdinalEncoder
trait_var = data.trait.copy()
ordinalenc = OrdinalEncoder()
data.SNP = ordinalenc.fit_transform(pd.DataFrame(data.SNP))
#data = data.dropna()
data = sm.add_constant(data.SNP)
model = sm.OLS(trait_var, data).fit()
model.summary()

In [ ]:
# This correctly calculates the paramters of the least square but I am not sure I want to continue on this path
data = pd.concat([hmp.T.iloc[2,:],trait.iloc[:,0]], axis=1).dropna()
A = np.vstack([data.iloc[:,0], np.ones(len(data))]).T
m, c = np.linalg.lstsq(A, data.iloc[:,1], rcond=None)[0]
print(c, m)

In [ ]:
# This also correctly calculates the paramters of the least square
data = pd.concat([hmp.T.iloc[2,:],trait.iloc[:,0]], axis=1).dropna()
data = sm.add_constant(data)
# https://github.com/tirthajyoti/Machine-Learning-with-Python/blob/e683fadd99ae35831fd65de1bb3569f8f751f448/Regression/Linear_Regression_Methods.ipynb
# https://www.freecodecamp.org/news/data-science-with-python-8-ways-to-do-linear-regression-and-measure-their-speed-b5577d75f8b/
# https://stackoverflow.com/questions/44659204/numpy-dot-product-with-missing-values
t = data.iloc[:,:2]
y = data.iloc[:,2]
m = np.dot((np.dot(np.linalg.inv(np.dot(t.T,t)),t.T)),y)
m

# 2. Calculate Distance Matrix

https://bitbucket.org/tasseladmin/tassel-5-source/wiki/UserManual/DistanceMatrix/DistanceMatrix \
https://davetang.org/muse/2015/07/24/dna-sequencing-data/ \
TASSEL calculates distance as 1 - IBS (identity by state) similarity, with IBS defined as the probability that alleles drawn at random from two individuals at the same locus are the same. For clustering, the distance of an individual from itself is set to 0.

The calculation is based on the definition. For a bi-allelic locus with alleles A and B, probabilityIBS(AA,AA) = 1, pIBS(AA,BB) = 0, pIBS(AB, xx) = 0.5, where xx is any other genotype. For two taxa, pIBS is averaged over all non-missing loci. Distance is 1 - pIBS. The kinship calculation is related but different and is described in Endelman and Jannink (2012) Shrinkage Estimation of the Realized Relationship Matrix. G3 2:1405-1413, using the non-shrunk version under the assumption that generally, number of markers > number of individuals.

Below is a python implementation of the IBM calculation using numpy vectorization. It assumes the hapmap file has only mono-allelic sites and only works for standard homozygous ("G", "C", "A", "T") and missing ("N") alleles.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# https://towardsdatascience.com/how-to-vectorize-pairwise-dis-similarity-metrics-5d522715fb4e
hmp = pd.read_csv("test_hapmap.txt", sep="\t", index_col=0)
hmp = hmp.iloc[:,10:]

# First step is to calculate all non-missing loci in a 2-d numpy array
X = hmp.copy()
X = X.replace(["G","C","A","T"],True)
X = X.replace("N",False)
X = np.array(X.T)
count_loc = np.empty((len(X), len(X)))
count_loc = (X[:, None, :]) & (X[None, :, :])
count_loc = count_loc.sum(axis=-1)

# Second step is to calculate all matching loci (np.nan==np.nan is false) in a 2-d numpy array
X = hmp.copy()
X = X.replace("N", np.nan)
X = np.array(X.T)
count_match = np.empty((len(X), len(X)))
count_match = X[:, None, :] == X[None, :, :]
count_match = count_match.sum(axis=-1)

# Third step is to calculate the 1 - IBS (IBS=matching/non-missing loci)
IBS = pd.DataFrame(1-(count_match/count_loc))
IBS.index = hmp.columns
IBS.columns = hmp.columns
IBS

In [ ]:
# This solution to calculate non-missing loci technically works but the
# addition of the two vectorized numpy arrays is very non-memory efficient
# (required about 80gb of RAM with just 13k SNPs in the hapmap file)
# custom function to count non-missing loci
def N_in(x):
    return len(x) - sum('N' in s for s in x)

X = np.array(X.T)
count_loc = np.empty((len(X), len(X)))
count_loc = X[:, None, :] + X[None, :, :]
count_loc = np.apply_along_axis(N_in, -1, count_loc)

In [ ]:
# This solution works but to calculate 1-IBS works but is slow due to nested loops
hmp = pd.read_csv("test_hapmap.txt", sep="\t", index_col=0)
hmp = hmp.iloc[:,10:]
hmp = hmp.replace("N", np.nan)
IBS  = pd.DataFrame(columns = hmp.columns, index=hmp.columns)
for ix1, row1 in hmp.T.iterrows():
    for ix2, row2 in hmp.T.iterrows():
        count_loc = (row1+row2).count()
        count_match = (row1==row2).sum()
        IBS.loc[ix1,ix2] = 1-count_match/count_loc